# Sensitivity Analysis: Morris Method

In [1]:
import numpy as np
import matplotlib.pyplot as plt

## 1. What is sensitivity analysis and why do we need it?

Explanation needed

## 2. Methods for sensitivity analysis.

Global, local, screening, variance-based, OAT, regression blah blah

## 3. Morris Method: a primer

## Sampling method

One of the key challenges in Morris' Elementary Effects method is: how can we create sample trajectories so that the input space is well-explored in an efficient way? In his original design, Morris suggested a method that creates r trajectories consisting of (k+1) points from the input space. Each trajectory can be used to calculate k elementary effects, one for each input factor. This section explains how we can obtain a set of these trajectories that allows for efficient evaluation of elementary effects.

### Choice of _p_ and ${\Delta}$

Choice of parameters _p_ and ${\Delta}$ are a critical part of the implementation of Morris method. 

Generally, an even value is recommended as the choice for p, as this simplifies the choice of ${\Delta}$. Assuming that p is equal, the delta is then chosen to be:

$$ {\Delta} = \frac{p}{2(p-1)}$$

According to Morris, this choice of ${\Delta}$ ensures that there is an equal probability of sampling from each level of the grid. Below is an implementation of the function to get delta:

In [2]:
def get_delta(num_levels):
    """
    Function for obtaining delta for Morris method.
    
    Parameters:
        num_levels - number of levels (p) of the input grid space.
    
    Returns:
        delta - increment to be used when obtaining elementary effects
    """
    delta = num_levels/(2*(num_levels-1))
    return delta

### x*

Each trajectory begins with a base vector (denoted as **x***) that is randomly selected from a p-level grid ${\Omega}$. The x* must be chosen in such a way so that after increasing one or more of the x* factors by ${\Delta}$ the point still lies within the input space.

In this implementation the grid from which we sample x* has boundaries of 0 and $(1 - {\Delta})$. The number of availaible points is $p / 2$, which is a common choice... (TO DO)

In [3]:
def get_x_star(num_levels, num_input, delta):
    """
    Function for obtaining delta for Morris method.
    
    Parameters:
        num_levels - number of levels (p) of the input grid space.
        num_input - number of input factors of the model
        delta - increment to be used when obtaining elementary effects
    
    Returns:
        x_star - basis vector / starting for the Morris trajectories
    """
    grid = np.linspace(0, 1-delta, int(num_levels/2))
    x_star = np.random.choice(grid, size=num_input)
    return x_star

### Trajectories

The idea behind the algorithm for generating a sampling trajectory is very simple: for a starting point (x*), a random factor is selected and this factor is incremented by $\pm{\Delta}$, generating a new point. This process is repeated until all factors have been moved once by ${\Delta}$.

A trajectory can be easily computed by considering it in a form of a matrix **B*** with dimensions of (k+1) x k, where each row is a point obtained by moving one factor of the model by ${\Delta}$. To construct **B***, we first choose a strictly lower triangular matrix of 1's, **B**, with dimensions of (k+1)x(k). This ensures that only one input factor is changed at a time, as for every column index j, there are two rows that only differ in the jth entry.

The sampling matrix is then given by:
    
where $J_{k+1}$ allows to create k+1 copies of the starting point $x^*$ of trajectory


A randomized version of this matrix is given by:

$J_{k+1,1}x^*$ : creates k+1 copies of point x*

$D^*$: determines whether factors move by $+{\Delta}$ or $-{\Delta}$

$P^*$ : a k-by-k random permutation matrix in which each row contains an element equal to 1 with all other elements equal to 0. As no two columns have 1s in the same position, this matrix determines the order in which the factors move.



In [4]:
def compute_Bstar(k, delta, x_star):
    # Generate matrix B
    B = np.tril(np.ones((k+1, k)), -1)
    # Generate matrix J
    J_copy = np.ones((k+1,1))
    J = np.ones((k+1, k))
    D_star = np.diag(np.random.choice([-1,1],size=k))
    P_star = np.eye(k,k)
    P_star = np.random.permutation(P_star)
    
    BmJ = 2*B-J
    DeltaBmJ = ((np.matmul(BmJ,D_star) + J)*delta/2)
    part1 = DeltaBmJ + J_copy*x_star
    B_star = np.matmul(part1, P_star)
    return B_star

We then repeat the process for the desired number of trajectories, generating our final Morris sample:

In [5]:
def morris_sample(k, n_trajectories, num_levels):
    delta = get_delta(num_levels)
    morris_sample = np.zeros([n_trajectories,k+1,k])
    for i in range(n_trajectories):
        x_star = get_x_star(num_levels, k, delta)
        morris_sample[i, :,:] = compute_Bstar(k, delta, x_star)
    return morris_sample

## Elementary Effects

Now that we have obtained our sample trajectories, majority of the hard work is done - we just need to evaluate our model on the sampled points and calculate the elementary effects. To recap, the elementary effect is defined as:

$$  EE_i=  \frac{[Y(X_1,X_2,…,X_{i-1},X_i+\Delta,…X_k )-Y(X_1,X_2,…,X_{i-1},X_i,…X_k )]}{\Delta}$$



Of course, looking at potentially 100s or 1000s elementary effects for each factor is probably not very informative. Morris thus proposed two sensitivity measures that can inform as about the effect of each factor.

The estimate of the **mean** of distribution of the elementary effects for each factor, ${\mu}$ , assesses the overall influence of the factor on the output of the model:
$${\mu_i} = \frac{1}{r}\sum_{j=1}^{r}EE_i^j$$
where $r$ is the number of trajectories.

The **standard deviation** of elementary effects of each factor, ${\sigma^2}$, estimates factor's nonlinear or interaction effect:
$${\sigma_i^2} = \frac{1}{r-1}\sum_{j=1}^{r}(EE_i^j-{\mu})$$

As sometimes the elementary effects might cancel each other out, decreasing the mean, Campologno et al. (2007) proposed an additional sensitivity measure - ${\mu^*}$, an estimate of the mean of distribution of the absolute elementary effects.
$${\mu_i} = \frac{1}{r}\sum_{j=1}^{r}\left |{EE_i^j}\right |$$

Below is my implementation of the elementary effects method. It calculates and returns all three sensitivity metrics:

In [6]:
def elementary_effects(sample, model, delta):
    
    y = np.zeros((sample.shape[0], sample.shape[1]))
    for i in range(sample.shape[0]):
        for j in range(sample.shape[1]):
            args = sample[i,j].tolist()
            y[i, j] = model(args)
    
    ee = np.zeros((sample.shape[0], sample.shape[2]))
    
    for i in range(sample.shape[0]):
        for j in range(sample.shape[1]-1):
            for k in range(sample.shape[2]):
                if sample[i, j+1, k] > sample[i,j,k]:
                    ee[i, k] = (y[i, j+1] - y[i,j])/delta
                elif sample[i, j+1, k] < sample[i,j,k]:
                    ee[i, k] = (y[i, j] - y[i,j+1])/delta
    
    ee_mean = [np.mean(ee[:,i]) for i in range(ee.shape[1])]
    ee_absmean = [np.mean(np.abs(ee[:,i])) for i in range(ee.shape[1])]
    ee_std = [np.std(ee[:,i]) for i in range(ee.shape[1])]
    
    return ee_mean, ee_absmean, ee_std

## Analysis of obtained results

We now have our summary statistics for elementary effects - but how do we interpret them? To show how one might perform a sensitivity analysis, we will test our implementation on a simple engineering function - Darcy's friction factor in terms of pressure loss and volumetric flow rate:

Just by looking at the above formulae, we can expect to obtain results that suggest the diameter of the pipe is most influential.

We start by generating our sampling trajectory:

We then have to scale our sampling trajectory - we assume nominal values of X and % variation.

Finally, we can calculate our sensitivity measures:

Plotting ${\sigma^2}$ versus ${\mu^*}$:

## Limitations

## Comparison with available software for SA

## 5. Morris method extensions